In [ ]:
## Multi Layer Neural network

In [ ]:
## in previous Notebook we have seen used single layer perceptron neural network and  Bi gram stats model both of them cannot capture the context better, they dont sound the names better
# so in this notebook we use the multilayer perceptron which are useful to generate the meaning full name

# so the problem with bi-gram model we face is we are able to capture the context of 2 characters at a time.(that gives us 27 character * 27 characters = 729 possibilities combination comes in context)
# if we have the 3 characters as context  we would get (27*27*27 =19,623 combinations context) generating count vector normalizing and everything becomes difficult now.
# the modeling approach we gonna follow  is from this research paper bengio et al. 2003 (https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)




In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [7]:
chars = sorted(list(set(''.join(words))))
stoi = {
    s: i+1 for i,s in enumerate(chars)
}
stoi['.'] = 0
itos = { i:s for s,i in stoi.items() }


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


## Creating dataset for Neural Networks from Text File to Input Tensor and Output Tensor

In [37]:
## data set for neural network

block_size = 3 #Context_length: how many characters do we take to predict the next one?
X,Y =[],[] # x are inputs and Y are labels

for w in words[:1]:

    print(w)
    context = [0] * block_size

    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)

        print("input data : ",''.join(itos[i] for i in context), ", Output :", '=', itos[ix])
        print('context :', context)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)


print('\n')
print("input tensor :", X)

print("output tensor :", Y)

emma
input data :  ... , Output : = e
context : [0, 0, 0]
input data :  ..e , Output : = m
context : [0, 0, 5]
input data :  .em , Output : = m
context : [0, 5, 13]
input data :  emm , Output : = a
context : [5, 13, 13]
input data :  mma , Output : = .
context : [13, 13, 1]


input tensor : tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1]])
output tensor : tensor([ 5, 13, 13,  1,  0])


In [ ]:
"""
Above output explanation

you can notes how the input data is store in the input tensor it is simple

input data :  ... , Output : = e
context : [0, 0, 0]

when we get input '...' we have the o/p as 'e'
similary i/p = '..e' => o/p = 'm'

-> we are trying to take context of 3 character at same time and trying to predict the o/p of what character come next and run in the loop

1)so we are able to use above block and change the parameters like block size to get different context
2) if we consider all the input and one cblock size we are able to get the whole i/p and o/p data



"""

# Block size = 5

In [38]:
block_size = 5 #Context_length: how many characters do we take to predict the next one?
X,Y =[],[] # x are inputs and Y are labels

for w in words[:1]:

    print(w)
    context = [0] * block_size

    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)

        print("input data : ",''.join(itos[i] for i in context), ", Output :", '=', itos[ix])
        print('context :', context)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)


print('\n')
print("input tensor :", X)

print("output tensor :", Y)

emma
input data :  ..... , Output : = e
context : [0, 0, 0, 0, 0]
input data :  ....e , Output : = m
context : [0, 0, 0, 0, 5]
input data :  ...em , Output : = m
context : [0, 0, 0, 5, 13]
input data :  ..emm , Output : = a
context : [0, 0, 5, 13, 13]
input data :  .emma , Output : = .
context : [0, 5, 13, 13, 1]


input tensor : tensor([[ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  5],
        [ 0,  0,  0,  5, 13],
        [ 0,  0,  5, 13, 13],
        [ 0,  5, 13, 13,  1]])
output tensor : tensor([ 5, 13, 13,  1,  0])


### above we are trying to take 5 character as input and predicting what comes next as o/p

In [87]:
block_size = 3 #Context_length: how many characters do we take to predict the next one?
X,Y =[],[] # x are inputs and Y are labels

for w in words[:5]:

    #print(w)
    context = [0] * block_size

    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)

        #print("input data : ",''.join(itos[i] for i in context), ", Output :", '=', itos[ix])
        #print('context :', context)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [88]:
import pandas as pd
pd.DataFrame(Y)

,0
0,5
1,13
2,13
3,1
4,0
5,15
6,12
7,9
8,22
9,9


In [89]:
"""
1. now the Test data with the block_size/context = 3 is ready and o/p is also ready fo the model to get trained on.
2.
"""

'\n1. now the Test data with the block_size/context = 3 is ready and o/p is also ready fo the model to get trained on.\n2.\n'

In [90]:
(X.shape, X.dtype), (Y.shape, Y.dtype)

((torch.Size([32, 3]), torch.int64), (torch.Size([32]), torch.int64))

In [91]:
## so our tensor matrix x has 228146 rows and 3 columns only both of there dtype is integer

## In paper they have 17,000 words and they embedded in 30 dimensional

## in our case we have only 27 possible characters so let's consider 2- dimensional space so we have 27 charcter with 2 dimenisonal space(27*2) tensor

In [92]:
# let embedded the 2 random numbers with 27*2

C = torch.randn((27,2)) # the weight Matrix is C it is represented as C because of research paper
C

tensor([[ 1.4055,  0.7574],
        [ 0.4587, -0.2725],
        [ 1.7137, -1.5466],
        [-1.7004, -0.1629],
        [-1.2717, -0.3085],
        [-0.1866,  1.1372],
        [ 0.1859, -0.9961],
        [-0.5628,  1.3539],
        [-0.5182,  0.8825],
        [-0.2112,  1.3888],
        [ 1.1893, -0.7963],
        [-0.4880, -1.0680],
        [ 0.1820,  0.4371],
        [-0.9799, -1.1722],
        [ 0.7273,  2.2199],
        [ 1.1401,  0.2118],
        [ 1.0594,  1.6841],
        [-1.7480, -0.4442],
        [-0.2313, -0.3345],
        [ 0.4041,  0.3533],
        [ 0.0263,  0.6341],
        [-0.4183, -0.9250],
        [-0.6277, -2.1561],
        [ 2.2610,  1.2614],
        [ 1.8530, -0.5818],
        [ 0.9229, -1.4164],
        [ 0.5225, -0.5810]])

In [93]:
C[5]

tensor([-0.1866,  1.1372])

In [94]:
#Example

(F.one_hot(torch.tensor(5), num_classes=27).float()) @ C

tensor([-0.1866,  1.1372])

In [95]:
# if we multiply the weight vector(C) * index_vector/one_hot encoding vector of the input(X) = we get the Result from the first layer

In [96]:
C = torch.randn((27,2))
C ## c is noting buit one type of encoding

tensor([[-1.3028,  1.6685],
        [-1.8216,  1.5519],
        [-0.6952, -0.2872],
        [-0.5589,  1.1143],
        [-1.3582, -0.5364],
        [-0.2447,  0.2761],
        [-0.7507,  1.7232],
        [ 0.3503, -1.4699],
        [-0.9065, -1.6470],
        [-0.3676,  1.0258],
        [-0.8222,  0.9258],
        [ 0.1590, -0.9919],
        [ 0.6491, -2.0204],
        [ 0.7483,  0.2963],
        [-0.4171, -0.4603],
        [-1.2425, -1.0488],
        [ 0.4187,  1.0417],
        [ 0.0578,  1.9587],
        [ 0.4469, -0.5635],
        [-1.1468, -0.5540],
        [ 0.2199, -1.3478],
        [-1.1349, -1.0798],
        [ 0.4670,  0.2617],
        [ 0.6398, -1.5389],
        [-0.1622,  1.3953],
        [-0.0336, -1.7348],
        [ 0.8888, -0.5408]])

In [97]:
# we want to encode one number 5 we can simply pass this number to the Embedding tensor C and get the output i.e.,

C[5] #Give embedding of 5 in 2d Vector Space

tensor([-0.2447,  0.2761])

In [98]:
# what if the embedding is like list

print(C[[2,3,4]]) # we got the embedding of 3 integer vector as below


tensor([[-0.6952, -0.2872],
        [-0.5589,  1.1143],
        [-1.3582, -0.5364]])


In [99]:
# what if we give the tensor input to get embedding

print(C[torch.tensor([5,6,7,7,7,7,7,7,])]) # still we are able to get the embedding by giving tensor matrix as input

tensor([[-0.2447,  0.2761],
        [-0.7507,  1.7232],
        [ 0.3503, -1.4699],
        [ 0.3503, -1.4699],
        [ 0.3503, -1.4699],
        [ 0.3503, -1.4699],
        [ 0.3503, -1.4699],
        [ 0.3503, -1.4699]])


In [100]:
"""
Full explanation of C

 C is the embedding vector  when you pass the single integer -> we get the embedding of that single integer
if we pass list to embedding vector c -> we goonna get embedding for that list passed
if we pass the tensor to embedding vector -. we gonna get th emebedding for that tensor


to get emebedding for anything we need to pass it to the C

"""

'\nFull explanation of C\n\n C is the embedding vector  when you pass the single integer -> we get the embedding of that single integer\nif we pass list to embedding vector c -> we goonna get embedding for that list passed\nif we pass the tensor to embedding vector -. we gonna get th emebedding for that tensor\n\n\nto get emebedding for anything we need to pass it to the C\n\n'

In [101]:
embedding = C[X]
embedding

tensor([[[-1.3028,  1.6685],
         [-1.3028,  1.6685],
         [-1.3028,  1.6685]],

        [[-1.3028,  1.6685],
         [-1.3028,  1.6685],
         [-0.2447,  0.2761]],

        [[-1.3028,  1.6685],
         [-0.2447,  0.2761],
         [ 0.7483,  0.2963]],

        [[-0.2447,  0.2761],
         [ 0.7483,  0.2963],
         [ 0.7483,  0.2963]],

        [[ 0.7483,  0.2963],
         [ 0.7483,  0.2963],
         [-1.8216,  1.5519]],

        [[-1.3028,  1.6685],
         [-1.3028,  1.6685],
         [-1.3028,  1.6685]],

        [[-1.3028,  1.6685],
         [-1.3028,  1.6685],
         [-1.2425, -1.0488]],

        [[-1.3028,  1.6685],
         [-1.2425, -1.0488],
         [ 0.6491, -2.0204]],

        [[-1.2425, -1.0488],
         [ 0.6491, -2.0204],
         [-0.3676,  1.0258]],

        [[ 0.6491, -2.0204],
         [-0.3676,  1.0258],
         [ 0.4670,  0.2617]],

        [[-0.3676,  1.0258],
         [ 0.4670,  0.2617],
         [-0.3676,  1.0258]],

        [[ 0.4670,  0

In [102]:
embedding.shape

torch.Size([32, 3, 2])

now our input is embeeded

In [103]:
# so to convert the input vector to 2 dimensional space there are manu ways to represent it
# but one of the effective and easiest way pass dotview  .view(x,6)

embedding.view(-1, 6)

tensor([[-1.3028,  1.6685, -1.3028,  1.6685, -1.3028,  1.6685],
        [-1.3028,  1.6685, -1.3028,  1.6685, -0.2447,  0.2761],
        [-1.3028,  1.6685, -0.2447,  0.2761,  0.7483,  0.2963],
        [-0.2447,  0.2761,  0.7483,  0.2963,  0.7483,  0.2963],
        [ 0.7483,  0.2963,  0.7483,  0.2963, -1.8216,  1.5519],
        [-1.3028,  1.6685, -1.3028,  1.6685, -1.3028,  1.6685],
        [-1.3028,  1.6685, -1.3028,  1.6685, -1.2425, -1.0488],
        [-1.3028,  1.6685, -1.2425, -1.0488,  0.6491, -2.0204],
        [-1.2425, -1.0488,  0.6491, -2.0204, -0.3676,  1.0258],
        [ 0.6491, -2.0204, -0.3676,  1.0258,  0.4670,  0.2617],
        [-0.3676,  1.0258,  0.4670,  0.2617, -0.3676,  1.0258],
        [ 0.4670,  0.2617, -0.3676,  1.0258, -1.8216,  1.5519],
        [-1.3028,  1.6685, -1.3028,  1.6685, -1.3028,  1.6685],
        [-1.3028,  1.6685, -1.3028,  1.6685, -1.8216,  1.5519],
        [-1.3028,  1.6685, -1.8216,  1.5519,  0.4670,  0.2617],
        [-1.8216,  1.5519,  0.4670,  0.2

In [104]:
## now if we want to pass it to the second layer

# consider second layer as weight w1 and bias b1
# then o/p of second layer = i/p @w1 + b1

#lets consider we have 100 weights in second layer such that we have 100 biases in second layer too h = tanh(w1*x1 + b1)

W1 = torch.rand((6,100))
B1 = torch.rand(100)

In [105]:
h = torch.tanh(embedding.view(-1, 6) @ W1 + B1)
h

tensor([[ 0.8388, -0.1336,  0.7544,  ...,  0.9729,  0.7419,  0.8800],
        [ 0.9551,  0.0895,  0.6731,  ...,  0.9686,  0.5060,  0.8940],
        [ 0.9318,  0.5509,  0.7485,  ...,  0.8910,  0.7745,  0.9290],
        ...,
        [ 0.2615, -0.9723, -0.2959,  ..., -0.5677, -0.9524, -0.4116],
        [-0.6253,  0.4373, -0.7034,  ..., -0.1920,  0.8582,  0.8393],
        [-0.4209, -0.9024, -0.7608,  ..., -0.5827, -0.9162, -0.5791]])

In [107]:
h.shape

torch.Size([32, 100])

In [109]:
"""
-> We are creating the output layer here -> output layer = weight * hidden layer + Bias

-> but we have only 27 character, so the layer needs to be 27 weights + 27 weights i.e., we need to take 100 inputs and give only 27 o/p

->


"""

W2 = torch.randn((100,27))
B2 = torch.randn(27)

In [111]:
logits = h @ W2+ B2
logits

tensor([[ 1.0882e+01,  1.0689e+01,  8.3067e+00, -1.0882e-01,  8.5998e+00,
          3.6865e+00,  1.2778e+01,  8.3188e+00,  6.1602e+00, -2.9399e+00,
          2.6398e+00,  2.2891e+00, -1.8251e+00, -2.0688e+00, -8.5253e+00,
         -5.1564e+00, -4.8787e+00,  8.0808e+00, -3.9176e+00,  6.1599e+00,
         -1.2264e+01, -3.5532e+00,  3.3294e+00, -1.9572e+00, -3.0141e+00,
         -6.7364e-01, -4.7768e+00],
        [ 8.9172e+00,  9.2561e+00,  9.5637e+00, -1.7320e+00,  5.9928e+00,
          7.5574e-01,  1.0065e+01,  5.0416e+00,  7.2937e+00, -2.6560e+00,
          2.9372e+00,  1.6444e+00, -4.1836e+00, -3.3501e+00, -7.1241e+00,
         -4.5624e+00, -5.4429e+00,  9.6493e+00, -3.5000e+00,  1.3368e+01,
         -1.2022e+01, -3.4132e+00, -2.6708e+00, -1.9486e+00, -2.9384e+00,
         -4.4481e+00, -3.8054e+00],
        [ 9.5722e+00,  8.4163e+00,  1.2724e+01, -7.7386e+00,  7.9873e+00,
         -1.7400e+00,  8.5007e+00,  6.1290e+00,  3.5561e+00,  6.5908e-01,
          3.4000e+00,  1.2511e+00, -5.82

In [112]:
counts = logits.exp()
counts

tensor([[5.3227e+04, 4.3853e+04, 4.0510e+03, 8.9689e-01, 5.4303e+03, 3.9905e+01,
         3.5451e+05, 4.1001e+03, 4.7354e+02, 5.2873e-02, 1.4010e+01, 9.8656e+00,
         1.6120e-01, 1.2634e-01, 1.9838e-04, 5.7621e-03, 7.6072e-03, 3.2319e+03,
         1.9889e-02, 4.7338e+02, 4.7183e-06, 2.8632e-02, 2.7922e+01, 1.4125e-01,
         4.9088e-02, 5.0985e-01, 8.4226e-03],
        [7.4589e+03, 1.0468e+04, 1.4239e+04, 1.7694e-01, 4.0054e+02, 2.1292e+00,
         2.3507e+04, 1.5471e+02, 1.4711e+03, 7.0230e-02, 1.8863e+01, 5.1782e+00,
         1.5244e-02, 3.5080e-02, 8.0545e-04, 1.0437e-02, 4.3271e-03, 1.5510e+04,
         3.0197e-02, 6.3938e+05, 6.0106e-06, 3.2934e-02, 6.9194e-02, 1.4247e-01,
         5.2949e-02, 1.1701e-02, 2.2251e-02],
        [1.4360e+04, 4.5201e+03, 3.3555e+05, 4.3567e-04, 2.9434e+03, 1.7553e-01,
         4.9181e+03, 4.5895e+02, 3.5027e+01, 1.9330e+00, 2.9964e+01, 3.4942e+00,
         2.9473e-03, 1.9847e-03, 1.4939e-04, 3.9163e-04, 2.1645e-02, 7.6004e+02,
         1.9085e-

In [114]:
probs  = counts/counts.sum(1, keepdims = True )
probs

tensor([[1.1338e-01, 9.3415e-02, 8.6294e-03, 1.9105e-06, 1.1567e-02, 8.5004e-05,
         7.5517e-01, 8.7340e-03, 1.0087e-03, 1.1263e-07, 2.9845e-05, 2.1015e-05,
         3.4339e-07, 2.6912e-07, 4.2258e-10, 1.2274e-08, 1.6205e-08, 6.8845e-03,
         4.2366e-08, 1.0084e-03, 1.0051e-11, 6.0991e-08, 5.9479e-05, 3.0089e-07,
         1.0457e-07, 1.0861e-06, 1.7942e-08],
        [1.0467e-02, 1.4690e-02, 1.9981e-02, 2.4829e-07, 5.6207e-04, 2.9879e-06,
         3.2988e-02, 2.1711e-04, 2.0643e-03, 9.8553e-08, 2.6470e-05, 7.2664e-06,
         2.1391e-08, 4.9227e-08, 1.1303e-09, 1.4646e-08, 6.0722e-09, 2.1765e-02,
         4.2376e-08, 8.9723e-01, 8.4346e-12, 4.6216e-08, 9.7099e-08, 1.9992e-07,
         7.4302e-08, 1.6420e-08, 3.1224e-08],
        [2.4517e-06, 7.7174e-07, 5.7290e-05, 7.4385e-14, 5.0254e-07, 2.9968e-11,
         8.3970e-07, 7.8360e-08, 5.9803e-09, 3.3004e-10, 5.1160e-09, 5.9659e-10,
         5.0320e-13, 3.3886e-13, 2.5506e-14, 6.6866e-14, 3.6956e-12, 1.2977e-07,
         3.2585e-

In [115]:
probs[torch.arange(32), Y]

tensor([8.5004e-05, 4.9227e-08, 3.3886e-13, 2.5331e-10, 1.0225e-02, 1.2274e-08,
        2.5989e-09, 2.2685e-09, 2.3873e-09, 1.1633e-08, 2.0684e-07, 1.5779e-02,
        9.3415e-02, 6.2441e-05, 1.6493e-03, 4.8448e-04, 1.1263e-07, 9.8020e-01,
        5.5473e-01, 1.1920e-06, 3.3058e-08, 4.2718e-09, 4.7727e-07, 9.5005e-09,
        1.1602e-13, 1.0084e-03, 3.0051e-07, 3.8790e-05, 1.1484e-07, 2.2878e-07,
        1.4225e-01, 2.4937e-10])

In [117]:
loss = -probs[torch.arange(32), Y].log().mean()
loss

tensor(13.7562)